**Step 1: Install necessary packages**


In [2]:
#Loading the environment
from dotenv import load_dotenv
import os

#For setting up agents
from phi.agent import Agent, RunResponse
from phi.model.openai import OpenAIChat
from phi.tools.exa import ExaTools
from phi.tools.newspaper4k import Newspaper4k

#For step 2: extracting regex
import re

#For step 3: Storing agent output
import json

#For step 4: Sending email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

**Step 2: Curate recipes from a specific website with criteria**


In [9]:
user_query = "Help me find vegan stir fry for kids. Absolutely no nuts."

In [10]:
curator_agent = Agent(
    description="you help user meal prep",
    name="Dabbawala",
    model=OpenAIChat(id="gpt-4o", api_key=openai_api_key),
    instructions=[
        "You are a highly knowledgeable meal prep planning.",
        "Your goal is to help the user identify recipes that satisfy dietary restrictions, preferences, and ingredient availability..",
        "Provide a short summary of each recipe recommended.",
        "Offer up to 5 recipes for each request, ensuring they are diverse and relevant.",
        "Leverage online resources like rainbowplantlife.com and minimalistbaker.com for accurate and varied suggestions. But feel free to search the entire internet",
        "Focus on being concise, relevant, and thoughtful in your recommendations.",
        "Recipes must be nut-free. There must also be some gluten-free options"
        "Always include sources"
    ],
    tools=[ExaTools()],
)
response: RunResponse = curator_agent.run(user_query)

INFO     Searching exa for: nut-free vegan stir fry recipe for kids

INFO     Searching exa for: child-friendly vegan stir fry nut-free

In [11]:
print(response.content)

Here are some nut-free vegan stir fry recipes perfect for kids:

1. **Rainbow Stir-Fry**:
   A colorful and easy-to-make stir fry featuring a variety of vegetables. It's a great way to get kids to eat veggies, as the vibrant colors often make it more appealing. [Source](https://blog.fatfreevegan.com/2006/12/rainbow-stir-fry.html)

2. **Easy Vegetable Stir Fry**:
   A versatile stir fry recipe that allows you to use any vegetables you have on hand. It's healthy, quick to make, and can be served either warm or cold. [Source](https://snotty-noses.com/blog/vegetable-stir-fry-quick-healthy-meal-kids-will-love)

3. **30 Min Healthy Chinese Vegetable Stir-fry**:
   A quick and clean recipe that uses organic ingredients and offers a gluten-free and soy-free option. It's packed with a variety of vegetables and can be served over rice. [Source](https://flexitariannutrition.com/meatless-monday-30-min-healthy-chinese-vegetable-stir-fry-vegan-soy-free-option-gluten-freepaleo/)

4. **Soy-free Tofu S

**Step 3: Extract URLs from the agent's recommendations**


In [12]:
url_pattern = r'https?://[^\)\s]+'
urls = re.findall(url_pattern, response.content)

**Step 4: Extract ingredient lists from URLs**

In [13]:
recipe_ingredients = {}
ingredient_agent = Agent(tools=[Newspaper4k()], debug_mode=False, show_tool_calls=False)

for url in urls:
    #print(f"Processing: {url}")
    query = f"Extract ingredient list from {url}. Make sure to not miss any ingredient and be exhaustive."
    response = ingredient_agent.run(query)
    recipe_ingredients[url] = response.content

# Save ingredients to a JSON file
with open('recipe_ingredients.json', 'w') as f:
    json.dump(recipe_ingredients, f, indent=4)

**Step 5: Send recipes via email**


In [14]:
# Email configuration
sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_APP_PASSWORD")  # Use app password for Gmail
receiver_email = os.getenv("RECEIVER_EMAIL")

# Read JSON data
with open('recipe_ingredients.json', 'r') as f:
    recipe_data = json.load(f)

# Create email content
def create_email_content():
    html_content = f"""
    <html>
        <body>
            <h2>Sous Chef Recommendations - {datetime.now().strftime('%Y-%m-%d')}</h2>
    """
    for url, ingredients in recipe_data.items():
        html_content += f"""
            <div style="margin-bottom: 30px;">
                <h3><a href="{url}">Recipe Link</a></h3>
                <div style="margin-left: 20px;">
                    {ingredients.replace('\n', '<br>')}
                </div>
            </div>
        """
    html_content += "</body></html>"
    return html_content

# Send the email
def send_email():
    if not sender_email or not sender_password or not receiver_email:
        raise ValueError("Missing required environment variables. Please set SENDER_EMAIL, SENDER_APP_PASSWORD, and RECEIVER_EMAIL.")

    msg = MIMEMultipart('alternative')
    msg['Subject'] = f"Sous Chef Recommends - {datetime.now().strftime('%Y-%m-%d')}"
    msg['From'] = sender_email
    msg['To'] = receiver_email

    html_content = create_email_content()
    msg.attach(MIMEText(html_content, 'html'))

    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as smtp_server:
            smtp_server.starttls()
            smtp_server.login(sender_email, sender_password)
            smtp_server.sendmail(sender_email, receiver_email, msg.as_string())
        print("Email sent successfully!")
    except smtplib.SMTPAuthenticationError as e:
        print(f"Authentication failed. Error: {e}")
    except Exception as e:
        print(f"Error sending email: {e}")

send_email()


Email sent successfully!
